In [1]:
%matplotlib notebook

from math import comb, log
import matplotlib.pyplot as plt
import numpy as np

# Traces of the Hecke Operator $T_3$

The following value of $N$ is the largest value of $k$ the notebook will attempt to compute.

In [10]:
N = 3000

In [11]:
a_k = [1, -2]
b_k = [1, 1]
c_k = [-1, 0, -9]
powers_of_2 = [1, 2]
powers_of_3 = [1, 3]

# Because of the upper bound we need to go to when computing, we need to compute these to a slightly larger value
for k in range(2, 2 * N):
    a_k.append(-5 * a_k[k-1] - 9 * a_k[k-2])
    b_k.append(-2 * b_k[k-1] - 9 * b_k[k-2])
    powers_of_2.append(2 * powers_of_2[k-1])
    powers_of_3.append(3 * powers_of_3[k-1])

for k in range(3, 2 * N):
    c_k.append(-27 * c_k[k-3])

In [12]:
binomial_coefficients = [[1]]
for n in range(1, 2 * N):
    last_row = binomial_coefficients[n-1]
    left_append = [0] + last_row
    right_append = last_row + [0]
    binomial_coefficients.append([right_append[k] + left_append[k] for k in range(n+1)])

In [7]:
def trace_3(two_k):
    k = two_k // 2
    total = 2 * pow(-3, k-2) - 1
    for j in range(k):
        if (j % 2 == 0):
            summand = 1
        else:
            summand = -1
            
        summand *= comb(2 * k - 2 - j, j)
        summand *= pow(3, j) + pow(3, j) * pow(2, 2 * k - 2 - 2 * j) + pow(3, 2 * k - 3 - j)
        total -= summand
    
    return total

def trace_3_fast(two_k):
    k = two_k // 2
    
    # First term
    total = 2 * powers_of_3[k-2]
    if k % 2 == 1:
        total *= -1
    total -= 1
    
    # The alternating sum
    for j in range(k):
        if (j % 2 == 0):
            summand = 1
        else:
            summand = -1

        summand *= binomial_coefficients[2 * k - 2 - j][j]
        summand *= powers_of_3[j] + powers_of_3[j] * powers_of_2[2 * k - 2 - 2 * j] + powers_of_3[2 * k - 3 - j]
        total -= summand

    return total

In [19]:
traces = [trace_3_fast(2 * k) for k in range(2, N)]

In [14]:
for k in range(2, N, 2):
    try:
        assert trace_3_fast(2 * k) == -1 - a_k[k-1] - b_k[k - 1] + c_k[k - 1]
    except AssertionError:
        print("k={} fails".format(k))

In [17]:
expand((x^2 + 5 * x + 9) * (x^2 + 2 * x + 9) * (x^3 + 27) * (x - 1))

x^8 + 6*x^7 + 21*x^6 + 62*x^5 + 180*x^4 + 486*x^3 + 945*x^2 + 486*x - 2187

In [32]:
for k in range(8, N-2):
    try:
        assert traces[k] == -6 * traces[k-1] - 21 * traces[k-2] - 62 * traces[k-3] - 180 * traces[k-4] - 486 * traces[k-5] - 945 * traces[k-6] - 486 * traces[k-7] + 2187 * traces[k-8]
    except AssertionError:
        print("k = {} fails".format(k))